# **SVD**

In [ ]:
# Full Scratch SVD
# ----------------

import math
import numpy as np
from collections import Counter
from sklearn.decomposition import TruncatedSVD

# -----------------------------
# Step 1: Preprocess documents
# -----------------------------
def preprocess(doc):
    return doc.lower().split()

# -----------------------------
# Step 2: TF and TF-IDF functions
# -----------------------------
def compute_tf(documents):
    """Returns TF matrix"""
    vocab = sorted(set(word for doc in documents for word in preprocess(doc)))
    vocab_index = {word: i for i, word in enumerate(vocab)}
    tf_matrix = np.zeros((len(vocab), len(documents)))

    for j, doc in enumerate(documents):
        words = preprocess(doc)
        word_counts = Counter(words)
        for word, count in word_counts.items():
            tf_matrix[vocab_index[word], j] = count / len(words)

    return tf_matrix, vocab

def compute_tfidf(documents):
    """Returns TF-IDF matrix"""
    vocab = sorted(set(word for doc in documents for word in preprocess(doc)))
    vocab_index = {word: i for i, word in enumerate(vocab)}
    N = len(documents)
    tfidf_matrix = np.zeros((len(vocab), N))

    # compute IDF
    idf = {}
    for word in vocab:
        df = sum(1 for doc in documents if word in preprocess(doc))
        idf[word] = math.log((N + 1) / (df + 1)) + 1  # smoothed IDF

    # compute TF-IDF
    for j, doc in enumerate(documents):
        words = preprocess(doc)
        word_counts = Counter(words)
        for word, count in word_counts.items():
            tf = count / len(words)
            tfidf_matrix[vocab_index[word], j] = tf * idf[word]

    return tfidf_matrix, vocab

# -----------------------------
# Step 3: Custom SVD via Eigen
# -----------------------------
def svd_from_scratch(A, k=None):
    # A is term-document matrix
    # Compute covariance matrices
    ATA = np.dot(A.T, A)   # document-document
    eigvals, eigvecs = np.linalg.eigh(ATA)  # eigen decomposition

    # Sort eigenvalues & vectors in descending order
    idx = np.argsort(-eigvals)
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]

    # Singular values are sqrt of eigenvalues
    singular_vals = np.sqrt(np.maximum(eigvals, 0))

    # Compute V and Sigma
    V = eigvecs
    Sigma = np.diag(singular_vals)

    # Compute U = A V Σ⁻¹
    U = np.dot(A, np.dot(V, np.linalg.inv(Sigma)))

    if k:  # Truncate
        U = U[:, :k]
        Sigma = Sigma[:k, :k]
        V = V[:, :k]

    return U, Sigma, V

# -----------------------------
# Step 4: Cosine similarity
# -----------------------------
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-9)

# -----------------------------
# Step 5: Example usage
# -----------------------------
if __name__ == "__main__":
    documents = [
        "Information retrieval is the process of obtaining information",
        "Machine learning improves information retrieval",
        "Deep learning advances machine intelligence"
    ]

    # Choose representation: TF or TF-IDF
    A, vocab = compute_tfidf(documents)  # or compute_tf(documents)
    print("Original Matrix (TF-IDF):")
    print(A)

    # Full SVD
    # U, S, VT = np.linalg.svd(X, full_matrices=False)
    U, S, V = svd_from_scratch(A, k=None)
    print("\nFull Reconstruction:")
    print(np.dot(U, np.dot(S, V.T)))

    # Truncated SVD (k=2)
    # svd = TruncatedSVD(n_components=2)
    # X_reduced = svd.fit_transform(X)
    U_k, S_k, V_k = svd_from_scratch(A, k=2)
    print("\nTruncated Reconstruction (k=2):")
    print(np.dot(U_k, np.dot(S_k, V_k.T)))

    # Query
    query = "information learning"
    query_vec = np.zeros((len(vocab),))
    words = preprocess(query)
    for word in words:
        if word in vocab:
            query_vec[vocab.index(word)] += 1
    # Project query into reduced space
    q_proj = np.dot(query_vec, U_k)
    docs_proj = np.dot(A.T, U_k)

    print("\nCosine Similarities (Query vs Docs in reduced space):")
    for i, doc_proj in enumerate(docs_proj):
        sim = cosine_similarity(q_proj, doc_proj)
        print(f"Doc {i+1}: {sim:.4f}")

# **MinHash**

In [ ]:
# MinHash using permutation
# -------------------------

import random
import numpy as np

# Example documents
docs = [
    "the cat sat on the mat",
    "the dog sat on the mat",
    "the cat chased the dog"
]

# Step 1: Shingling (k=2 or 3)
def get_shingles(doc, k=2):
    words = doc.split()
    return {" ".join(words[i:i+k]) for i in range(len(words)-k+1)}

k = 2
shingle_set = set()
doc_shingles = []

for d in docs:
    sh = get_shingles(d, k)
    doc_shingles.append(sh)
    shingle_set |= sh

shingles = list(shingle_set)

# Step 2: Build binary shingle–document matrix
matrix = []
for sh in shingles:
    row = [1 if sh in doc_shingles[j] else 0 for j in range(len(docs))]
    matrix.append(row)

sd_matrix = np.array(matrix)

print("Shingle–Document Matrix:")
print(sd_matrix)

# Step 3: MinHash Implementation
num_shingles, num_docs = sd_matrix.shape
num_hashes = 5  # number of permutations
signature = np.full((num_hashes, num_docs), np.inf)

rows = list(range(num_shingles))
permutations = [random.sample(rows, len(rows)) for _ in range(num_hashes)]

for i, perm in enumerate(permutations):
    for col in range(num_docs):
        for row in perm:
            if sd_matrix[row, col] == 1:
                signature[i, col] = row
                break

print("\nSignature Matrix:")
print(signature)

# Step 4: Similarity from signatures
def minhash_sim(col1, col2):
    return np.mean(signature[:, col1] == signature[:, col2])

print("\nSimilarity between Doc1 & Doc2:", minhash_sim(0, 1))
print("Similarity between Doc1 & Doc3:", minhash_sim(0, 2))
print("Similarity between Doc2 & Doc3:", minhash_sim(1, 2))










# MinHash using hash function
# ---------------------------

import random
import numpy as np

# Example documents
docs = [
    "the cat sat on the mat",
    "the dog sat on the mat",
    "the cat chased the dog"
]

# Step 1: Shingling (k=2 or 3)
def get_shingles(doc, k=2):
    words = doc.split()
    return {" ".join(words[i:i+k]) for i in range(len(words)-k+1)}

k = 2
shingle_set = set()
doc_shingles = []

for d in docs:
    sh = get_shingles(d, k)
    doc_shingles.append(sh)
    shingle_set |= sh

shingles = list(shingle_set)
num_shingles = len(shingles)

# Step 2: Build binary shingle–document matrix
matrix = []
for sh in shingles:
    row = [1 if sh in doc_shingles[j] else 0 for j in range(len(docs))]
    matrix.append(row)

sd_matrix = np.array(matrix)

print("Shingle–Document Matrix:")
print(sd_matrix)

# Step 3: MinHash using hash functions
num_shingles, num_docs = sd_matrix.shape
num_hashes = 5  # number of hash functions
signature = np.full((num_hashes, num_docs), np.inf)

# Create random hash functions of form: h(x) = (a*x + b) % p
p = 2 * num_shingles  # prime > num_shingles
hash_funcs = [(random.randint(1, p-1), random.randint(0, p-1)) for _ in range(num_hashes)]

for i, (a, b) in enumerate(hash_funcs):
    for row in range(num_shingles):
        hash_val = (a * row + b) % p
        for col in range(num_docs):
            if sd_matrix[row, col] == 1:
                if hash_val < signature[i, col]:
                    signature[i, col] = hash_val

print("\nSignature Matrix:")
print(signature.astype(int))

# Step 4: Similarity from signatures
def minhash_sim(col1, col2):
    return np.mean(signature[:, col1] == signature[:, col2])

print("\nSimilarity between Doc1 & Doc2:", minhash_sim(0, 1))
print("Similarity between Doc1 & Doc3:", minhash_sim(0, 2))
print("Similarity between Doc2 & Doc3:", minhash_sim(1, 2))










# Cosine similarity And Euclidean Case
# ------------------------------------

import numpy as np
from numpy.linalg import norm

# Example documents
docs = [
    "this is a cat",
    "this is a dog",
    "cats and dogs are animals",
    "the dog chased the cat"
]

# Step 1: Vocabulary
words = list(set(" ".join(docs).split()))

# Step 2: Term Frequency (TF)
tf = []
for d in docs:
    row = [d.split().count(w) for w in words]
    tf.append(row)
tf = np.array(tf)

# Step 3: Inverse Document Frequency (IDF)
N = len(docs)
idf = np.log(N / (np.count_nonzero(tf, axis=0)))

# Step 4: TF-IDF
tfidf = tf * idf
print("TF-IDF Matrix:\n", tfidf)

# Step 5: Pairwise Cosine Similarity and Euclidean Distance
num_docs = len(docs)

print("\nCosine Similarities:")
for i in range(num_docs):
    for j in range(i+1, num_docs):
        cosine_sim = np.dot(tfidf[i], tfidf[j]) / (norm(tfidf[i]) * norm(tfidf[j]))
        print(f"Doc{i+1} vs Doc{j+1}: {cosine_sim:.4f}")

print("\nEuclidean Distances:")
for i in range(num_docs):
    for j in range(i+1, num_docs):
        euclidean_dist = norm(tfidf[i] - tfidf[j])
        print(f"Doc{i+1} vs Doc{j+1}: {euclidean_dist:.4f}")

In [ ]:
import numpy as np
import math
import random

# ------------------ MinHash Part ------------------

def build_shingle_matrix(docs, k=2):
    """
    Build binary shingle-document matrix using k-word shingles.
    """
    shingles = set()
    doc_shingles = []

    for doc in docs:
        words = doc.lower().split()
        # Create k-word shingles
        s = {" ".join(words[i:i+k]) for i in range(len(words) - k + 1)}
        shingles |= s
        doc_shingles.append(s)

    shingles = list(shingles)
    shingle_index = {s: i for i, s in enumerate(shingles)}
    # {'quick brown dog': 0, 'fast brown fox': 1, 'the fast brown': 2, 'quick brown fox': 3, 'the quick brown': 4}
    # print(shingle_index)

    # Binary matrix
    M = np.zeros((len(shingles), len(docs)), dtype=int)
    for j, s_set in enumerate(doc_shingles):
        for s in s_set:
            M[shingle_index[s], j] = 1

    return M, shingles



def minhash_signature(M, num_hashes=100):
    """
    Generate signature matrix using MinHash.
    """
    n_shingles, n_docs = M.shape
    sig = np.full((num_hashes, n_docs), np.inf)

    # Random hash functions: (a*x + b) % p
    p = 2**31 - 1
    hash_funcs = [(random.randint(1, p-1), random.randint(0, p-1)) for _ in range(num_hashes)]

    for r in range(n_shingles):
        row = M[r]
        for h, (a, b) in enumerate(hash_funcs):
            hash_val = (a*r + b) % p
            for c in range(n_docs):
                if row[c] == 1:
                    sig[h, c] = min(sig[h, c], hash_val)
    return sig

def minhash_signature_permutation(M, num_permutations=100):
    """
    Generate signature matrix using MinHash with random row permutations.
    """
    n_shingles, n_docs = M.shape
    sig = np.full((num_permutations, n_docs), np.inf)

    for p in range(num_permutations):
        # Create a random permutation of rows
        perm = np.random.permutation(n_shingles)
        for c in range(n_docs):
            # Find the first row (in perm order) where doc has a 1
            for idx in perm:
                if M[idx, c] == 1:
                    sig[p, c] = idx
                    break
    return sig

def jaccard_from_signatures(sig, i, j):
    """
    Approximate Jaccard similarity between documents i and j.
    """
    return np.mean(sig[:, i] == sig[:, j])


# ------------------ TF-IDF Part ------------------

def tokenize(doc):
    return doc.lower().split()

def build_count_matrix(docs):
    """
    Count matrix (term-document).
    """
    vocab = {}
    tokenized_docs = []
    for doc in docs:
        tokens = tokenize(doc)
        tokenized_docs.append(tokens)
        for t in tokens:
            if t not in vocab:
                vocab[t] = len(vocab)

    V = len(vocab)
    D = len(docs)
    count_matrix = np.zeros((V, D), dtype=float)

    for j, tokens in enumerate(tokenized_docs):
        for t in tokens:
            i = vocab[t]
            count_matrix[i, j] += 1

    return count_matrix, vocab

def compute_tfidf(count_matrix):
    """
    Compute TF-IDF matrix manually.
    """
    V, D = count_matrix.shape
    tfidf = np.zeros((V, D))

    # Document frequencies
    df = np.count_nonzero(count_matrix > 0, axis=1)

    for j in range(D):
        for i in range(V):
            tf = count_matrix[i, j] / (np.sum(count_matrix[:, j]) + 1e-10)
            idf = math.log((D + 1) / (df[i] + 1)) + 1
            tfidf[i, j] = tf * idf

    return tfidf

def cosine_similarity(A):
    """
    Compute cosine similarity matrix for document vectors.
    """
    D = A.shape[1]
    sim = np.zeros((D, D))
    for i in range(D):
        for j in range(D):
            num = np.dot(A[:, i], A[:, j])
            denom = (np.linalg.norm(A[:, i]) * np.linalg.norm(A[:, j]) + 1e-10)
            sim[i, j] = num / denom
    return sim

def euclidean_distance(A):
    """
    Compute Euclidean distance matrix for document vectors.
    """
    D = A.shape[1]
    dist = np.zeros((D, D))
    for i in range(D):
        for j in range(D):
            dist[i, j] = np.linalg.norm(A[:, i] - A[:, j])
    return dist


# ------------------ Example Usage ------------------

if __name__ == "__main__":
    docs = [
        "the quick brown fox",
        "the quick brown dog",
        "the fast brown fox"
    ]

    print("\n=== MinHash (Binary Shingle Matrix) ===")
    M, shingles = build_shingle_matrix(docs, k=3)
    print("Binary Shingle-Doc Matrix:\n", M)

    sig = minhash_signature(M, num_hashes=50)

    # Hash function method
    for i in range(len(docs)):
        for j in range(i+1, len(docs)):
            sim = jaccard_from_signatures(sig, i, j)
            print(f"Estimated Jaccard similarity (Doc {i}, Doc {j}): {sim:.3f}")


    # Permutation method
    sig_perm = minhash_signature_permutation(M, num_permutations=50)
    print("\n--- Using Permutation Method ---")
    for i in range(len(docs)):
        for j in range(i+1, len(docs)):
            sim = jaccard_from_signatures(sig_perm, i, j)
            print(f"Estimated Jaccard similarity (Doc {i}, Doc {j}): {sim:.3f}")



    print("\n=== TF-IDF (Cosine & Euclidean) ===")
    count_matrix, vocab = build_count_matrix(docs)
    tfidf = compute_tfidf(count_matrix)

    cos_sim = cosine_similarity(tfidf)
    euc_dist = euclidean_distance(tfidf)

    print("Cosine Similarity:\n", cos_sim)
    print("Euclidean Distances:\n", euc_dist)

# **PageRank**

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

def pagerank(adj_matrix, alpha=0.15, tol=1e-6, max_iter=100):
    """
    PageRank using power iteration:
    PR(k+1) = PR(k) * P
    """
    n = adj_matrix.shape[0]

    # Step 1: Build Hyperlink matrix H
    H = adj_matrix.astype(float)
    row_sums = H.sum(axis=1)

    for i in range(n):
        if row_sums[i] != 0:
            H[i] /= row_sums[i]
        else:
            # Dangling node → distribute uniformly
            H[i] = np.ones(n) / n

    # Step 2: Build transition matrix P
    P = (1 - alpha) * H + alpha * np.ones((n, n)) / n

    # Step 3: Initialize PageRank vector
    PR = np.ones(n) / n

    # Step 4: Power iteration
    for _ in range(max_iter):
        new_PR = PR @ P
        if np.linalg.norm(new_PR - PR, 1) < tol:
            PR = new_PR
            break
        PR = new_PR
        # print(PR)
    return PR / PR.sum()  # Normalize

def pagerank_eigen(adj_matrix, alpha=0.15):
    """
    PageRank using eigenvector method.
    """
    n = adj_matrix.shape[0]

    # Step 1: Build Hyperlink matrix H
    H = adj_matrix.astype(float)
    row_sums = H.sum(axis=1)
    for i in range(n):
        if row_sums[i] != 0:
            H[i] /= row_sums[i]
        else:
            H[i] = np.ones(n) / n

    # Step 2: Build transition matrix P
    P = (1 - alpha) * H + alpha * np.ones((n, n)) / n

    # Step 3: Solve eigenvector problem for P^T
    vals, vecs = np.linalg.eig(P.T)

    # Find index of eigenvalue closest to 1
    idx = np.argmin(np.abs(vals - 1))

    # Get corresponding eigenvector
    principal = np.real(vecs[:, idx])

    # Normalize to make it a probability distribution
    pr = principal / principal.sum()
    return pr

def plot_graph(adj_matrix):
    # Create directed graph
    G = nx.DiGraph(adj)

    # Compute PageRank using NetworkX (built-in)
    pr = nx.pagerank(G, alpha=0.85)
    print(pr)

    # Node sizes proportional to PageRank
    node_sizes = [5000 * pr[node] for node in G.nodes()]

    # Draw graph
    pos = nx.spring_layout(G, seed=42)
    nx.draw(G, pos, with_labels=True, node_size=node_sizes,
            node_color="skyblue", arrows=True, font_weight="bold")

    plt.show()

# Example usage
if __name__ == "__main__":
    adj = np.array([
        [0, 1, 1, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 1],
        [0, 0, 1, 0]
    ])

    pr = pagerank(adj, alpha=0.15)
    print("PageRank:", pr)

    # Ranking documents
    ranking = np.argsort(-pr)  # sort in descending order
    print("Ranking of documents (best to worst):", ranking)

    plot_graph(adj)

In [ ]:
#Page Rank using Power iteration
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt


def pagerank_eigen(adj_matrix, alpha=0.85):
    n = adj_matrix.shape[0]

    # Step 1: Construct hyperlink matrix H
    H = np.zeros((n, n))
    for i in range(n):
        out_links = np.sum(adj_matrix[i])
        if out_links > 0:
            H[i] = adj_matrix[i] / out_links

    # Step 2: Fix dangling nodes
    for i in range(n):
        if np.sum(H[i]) == 0:
            H[i] = np.ones(n) / n

    # Step 3: Google matrix
    G = alpha * H + (1 - alpha) * (np.ones((n, n)) / n)

    # Step 4: Eigen decomposition
    eigvals, eigvecs = np.linalg.eig(G.T)

    # Find the index of eigenvalue closest to 1
    idx = np.argmin(np.abs(eigvals - 1))

    # Corresponding eigenvector
    principal_eigvec = np.real(eigvecs[:, idx])

    # Normalize to sum to 1
    pagerank = principal_eigvec / np.sum(principal_eigvec)

    return pagerank


def pagerank(adj_matrix, alpha=0.85, tol=1e-6, max_iter=100):

    n = adj_matrix.shape[0]

    # Step 1: Construct hyperlink matrix H
    H = np.zeros((n, n))
    for i in range(n):
        out_links = np.sum(adj_matrix[i])
        if out_links > 0:
            H[i] = adj_matrix[i] / out_links

    # Step 2: Fix dangling nodes
    for i in range(n):
        if np.sum(H[i]) == 0:
            H[i] = np.ones(n) / n

    # Step 3: Google matrix
    G = alpha * H + (1 - alpha) * (np.ones((n, n)) / n)

    # Step 4: Power iteration
    rank = np.ones(n) / n
    for _ in range(max_iter):
        new_rank = rank @ G
        if np.linalg.norm(new_rank - rank, 1) < tol:
            break
        rank = new_rank

    return rank


# ====== Generate Graph ======
# Example: Directed graph
G = nx.DiGraph()

# Add edges (you can modify as needed)
edges = [
    (1, 2), (2, 5), (3, 1), (3, 2), (3, 4), (3, 5),
    (4, 5), (5, 4)
]
G.add_edges_from(edges)

# Draw graph
plt.figure(figsize=(6, 6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_size=1000, node_color="skyblue", font_size=12, arrows=True)
plt.title("Directed Graph")
plt.show()

# ====== Create adjacency matrix ======
adj_matrix = nx.to_numpy_array(G, nodelist=sorted(G.nodes()), dtype=int)
print("Adjacency Matrix:\n", adj_matrix)

# ====== Run PageRank ======
pagerank_scores = pagerank(adj_matrix)
print("PageRank Scores:", pagerank_scores)
print("Sum of scores (should be 1):", np.sum(pagerank_scores))